In [21]:
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup
from urllib.parse import urlparse, urljoin
import re

##dbs星展, sinopac永豐
bank_urls = {
            'dbs':'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/default.page', 
            'sinopac':'https://bank.sinopac.com/sinopacBT/personal/credit-card/introduction/list.html',
            'cathy': 'https://www.cathaybk.com.tw/cathaybk/personal/product/credit-card/cards/',
            'firstbank': 'https://card.firstbank.com.tw/sites/card/CreditCardList'
             }
credit_card_urls = {}
soup_documents = {k:None for k in list(bank_urls.keys())}

for bank, url in bank_urls.items():
    loader = RecursiveUrlLoader(url=url)
    docs = loader.load()
    if bank == 'dbs':
        links = list(set([a['href'] for a in Soup(docs[0].page_content).find_all('a', href=True) if (a['href'].startswith('/personal-zh/cards')) and (a['href'].endswith('hyperlink'))]))
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'sinopac':
        links = list(set([a['href'] for a in Soup(docs[0].page_content).find_all('a', href=True) if a['href'].startswith('./')]))
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'firstbank':
        def firstbank_extractor(html:str):
            soup_obj = Soup(html, 'html.parser')
            credit_card_feature = [re.sub(r'\n+', '\n', d.text) for d in soup_obj.find_all('div', {"class":[["card-single-features"]]})]
            txt = ''
            for features in  credit_card_feature:
                split = features.split('\n', 1)
                txt += f"信用卡卡名：{split[0].strip()}\n信用卡特色: {split[1]}"
            return txt
        all_html = RecursiveUrlLoader(bank_urls['firstbank'], extractor=firstbank_extractor).load()
        soup_documents[bank] = all_html

        divs = [Soup(doc.page_content, 'html.parser').find_all('div', {"class":'card-single'}) for doc in docs][0]
        links = []
        for d in divs:
            link_tags = d.find_all('a')
            if len(link_tags) == 1:
                detail_url = link_tags[0]['href']
                links.append(detail_url)
            else:
                for tag in link_tags:
                    if tag.text == '詳細內容':
                        links.append(tag['href'])
                    else:
                        continue
        credit_card_urls[bank] = [urljoin(url, i) for i in links]
    elif bank == 'cathy':
        def parser(html):
            ## main page
            divs = Soup(html, 'html.parser').find_all('div', {'class':'cubre-m-compareCard -credit'})
            if divs:
                divs = [re.sub(r'\n+', '\n', i.text) for i in divs]
                txt = (' ').join(divs)
                txt = re.sub(r' +', ' ', txt)
                txt = [t.replace('\n立即申辦','', 1) for t in txt.split('詳細說明') if t.startswith('\n立即申辦')]
                txt = ('').join(txt).replace('\n \n', '\n')
            else:
                ## deeper page
                divs = Soup(html, 'html.parser').find_all('div', class_=["cubre-o-textContent", "cubre-m-colorBanner__title","cubre-m-iconEssay__title","cubre-m-horGraphic__title","cubre-m-remind__title","cubre-m-puzzle__title","cubre-a-kvTitle -card"])
                divs = [d.text for d in divs if '您將離開本行官網 前往外部網站' not in d.text]
                uni_divs = []
                for d in divs[:len(divs)-1]:
                    if d not in uni_divs:
                        uni_divs.append(d)
                txt = re.sub(r'\n+', '\n', ('\n').join(uni_divs))
                txt = re.sub(r' +', ' ', txt)
            return txt
            
        docs = RecursiveUrlLoader(url=url, extractor=parser).load()
        for d in docs:
            d.metadata['bank'] = '國泰銀行'
            d.metadata['credit card'] = d.metadata['title'].split('-', 1)[0].strip()
        docs = [d for d in docs if '停發' not in d.metadata['title']]
        soup_documents['cathy'] = docs
        

In [4]:
from langchain_community.document_loaders.url_selenium import SeleniumURLLoader
# documents = {}
# for bank, links in credit_card_urls.items():
#     documents[bank] = SeleniumURLLoader(urls=links).load()
        

In [ ]:
def sinopac_extractor(html: str) -> str:
    soup = Soup(html, "html.parser")
    divs_txt = [s.text for s in soup.find_all('div', {'class':'tab-box'})]
    divs_txt.insert(0, re.sub(r'\n\n+', '\n', soup.find('div', {'class':'info'}).text))
    div_set = []
    for txt in divs_txt:
        if txt not in div_set:
            div_set.append(txt)
    txt = re.sub(r'[\t\r\xa0]', '', ('\n').join(div_set))
    txt = re.sub(r'  +', ' ', txt)
    txt = re.sub(r'\n+', '\n', txt)
    return txt

def dbs_extractor(html:str)->str:
    soup = Soup(html, "html.parser")
    divs_txt = [s.text for s in soup.find_all('div', {'class':'flpweb-legacy'})]
    if divs_txt:
        div_set = []
        for txt in divs_txt:
            if txt not in div_set:
                div_set.append(txt)
        all_txt = ('\n').join(div_set)
    else:
        txt = soup.text
        txt = [t.strip() for t in txt.split('\n') if( t != 'more..') and (len(txt.strip()) != 1)]
        all_txt = ('\n').join(txt)

    all_txt = all_txt.replace('個人網路銀行\nCard+ 信用卡數位服務\n企業網路銀行', '')
    all_txt = re.sub(r'[\t\r\xa0]', '', all_txt)
    all_txt = re.sub(r'\xa0', '', all_txt)
    all_txt = re.sub(r'  +', ' ', all_txt)
    all_txt = re.sub(r'\n+', '\n', all_txt)
    all_txt = all_txt.split('信用卡刷卡優惠更多星展信用卡刷卡優惠信息，詳情請見刷卡優惠說明。')[0]

    if '假字假字假字假字假字假字假字假字假字假字' in all_txt:
        all_txt = re.sub('假字假字假字+', '', all_txt)

    pattern = r"網銀登入\n[\s\S]*?選擇您的網站 ▼\n個人金融\n個人金融\n財富管理\n星展豐盛理財\n星展豐盛私人客戶\n企業金融\n中小企業銀行\n企業及機構銀行\n星展集團\n關於我們"
    # Replace the matched portion with an empty string
    all_txt = re.sub(pattern, '', all_txt)
    pattern = r'Loading...\n尊享至上\n[\s\S]*?\n權益手冊\n更多獨享優惠'
    all_txt = re.sub(pattern, '', all_txt)
    pattern = r'新戶好禮\n[\s\S]*?\n更多星展信用卡'
    all_txt = re.sub(pattern, '', all_txt)
    all_txt = all_txt.split('謹慎理財 信用至上')[0]
    return all_txt

def firstbank_extractor(html:str)->str:
    divs = Soup(html, 'html.parser').find_all('div', {"class":['carousel-item', 'content-body']})
    return ('').join([re.sub(r'\n+', '\n', d.text) for d in divs])

for bank, links in credit_card_urls.items():
    docs = []
    if bank == 'sinopac':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=sinopac_extractor).load()
            doc[0].metadata['bank'] = '永豐銀行'
            docs.extend(doc)
    elif bank == 'dbs':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=dbs_extractor).load()
            if doc:
                doc[0].metadata['bank'] = '星展銀行'
                docs.extend(doc)
    elif bank == 'firstbank':
        for url in links:
            doc = RecursiveUrlLoader(url=url, extractor=firstbank_extractor).load()
            doc[0].metadata['bank'] = '第一銀行'
            credit_card = doc[0].metadata['title'].split('-', 1)[0].strip()
            if '卡' in credit_card:
                doc[0].metadata['credit card'] = credit_card
            else:
                doc[0].metadata['credit card'] = ''
            docs.extend(doc)
    if soup_documents[bank]:
        soup_documents[bank].extend(docs)
    else:
        soup_documents[bank] = docs

In [47]:
soup_documents['dbs'][1].metadata

{'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/livefresh?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-hyperlink',
 'content_type': 'text/html',
 'title': '星展炫晶御璽卡 | 台灣星展銀行',
 'description': '海外消費2.52%／國內消費1.2%現金回饋，不只生活更要炫活',
 'language': None,
 'bank': '星展銀行'}

In [23]:
for k,v in soup_documents.items():
    print(f"Bank {k}: {len(v)}")

Bank dbs: 23
Bank sinopac: 46
Bank cathy: 17
Bank firstbank: 37


## Extract credit card name

In [64]:
soup_documents['sinopac'][5].metadata,soup_documents['firstbank'][3].metadata['bank'],soup_documents['firstbank'][3].metadata['description']

({'source': 'https://bank.sinopac.com/sinopacBT/personal/credit-card/introduction/co-brand/melaleuca-card.html',
  'content_type': 'text/html',
  'title': '美樂家生活卡 – 永豐銀行',
  'description': '美樂家生活卡專享每筆消費最高0.6%現金回饋! 是一張處處為您精打細算的信用卡!讓您所消費的每一分錢都值得輕鬆擁有健康、自在、富足的健康生活。',
  'language': 'zh-tw',
  'bank': '永豐銀行'},
 '第一銀行',
 '一銀百年來與台灣社會、經濟一起成長、茁壯，一貫秉持「顧客至上，服務第一」的經營理念，爲客戶的財富與託付創造更多的價值。期許能成為活躍亞洲的區域型銀行；予客戶最滿意之金融服務，予股東最豐碩且穩健之獲利，予員工最佳的生涯發展空間。')

In [69]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
import string


extractor = ChatOllama(model='qwen2:1.5b', temperature=0)
extract_prompt = PromptTemplate(template="""請幫我提取以下文字內的信用卡名稱，輸出格式請以json格式輸出，輸出格式請遵守{{"credit card":"xxxx"}}

範例:
文字:星巴克免費升級植物奶 日本／韓國／歐洲／美洲／新加坡最優5%回饋 | 星展銀行（台灣）
答案: {{"credit card":[]}}                            


文字: {text}
答案:
""", input_variables=['text', 'bank'])

chain = extract_prompt | extractor | JsonOutputParser()
chinese_punctuation = "。！；：，、（）【】《》『』——"
translator = str.maketrans('', '', chinese_punctuation)

for k in list(soup_documents.keys()):
    if( k == 'dbs') or (k == 'sinopac'):
        for doc in soup_documents[k]:
            try:
                credit_card = doc.metadata['title'].split('|', 1)[0].strip()
                if ('卡' in credit_card) and (credit_card.endswith('卡')):
                    doc.metadata['credit card'] = credit_card
                else:
                    result = chain.invoke({'text':doc.metadata['title'].split('|', 1)[0], 'bank':doc.metadata['bank']})
                    if result['credit card']:
                        cleaned_text = result['credit card'].translate(translator)
                        doc.metadata['credit card'] = cleaned_text
                    else:
                        doc.metadata['credit card'] = ''
                        print("No credit card found.")
                        print(doc.metadata['title'])
            except Exception as e:
                print("Error: ", e)
                print(doc.metadata['title'])
    # elif k == 'sinopac':
    #     for doc in soup_documents[k]:
    #         credit_card = doc.metadata['title'].split('-', 1)[0].strip()

No credit card found.
現金回饋國外2%、國內1%


In [71]:
[d.metadata['credit card'] for d in soup_documents['sinopac']]

['永豐銀行',
 '钛丰卡 - 永豐銀行',
 '永豐銀行',
 '55688聯名卡',
 '點字金融信用卡, 点字combo 卡, 永豐點字卡, 点字卡',
 '美樂家生活卡 – 永豐銀行',
 '永豐幣倍卡',
 '雄獅T&T白金卡 – 永豐銀行',
 '永豐銀行',
 '商务御璽數位企業卡–永豐銀行',
 '欣興 – 永豐銀行',
 '永豐保倍卡-永豐銀行',
 '永豐世界卡-永豐銀行',
 '商务御璽公司卡 – 永豐銀行',
 '雙幣公司卡– 永豐銀行',
 'DAWAY卡',
 '美安聯名卡-永豐銀行',
 'Vogue Card - Yongfeng Bank',
 '永傳世界卡',
 '其他信用卡 – 永豐銀行',
 '永豐SPORT卡',
 '',
 '永豐銀行',
 '現金回饋無上限國外2%國內1%終身免年費',
 'Google Pay - 永豐銀行',
 '永豐銀行',
 '永豐銀行',
 '雙獅旅天下白金卡 – 永豐銀行',
 '永豐銀行',
 '中興大學悠遊認同卡 – 永豐銀行',
 'SPORT Debit – 永豐銀行',
 '永富世界卡',
 '夢行卡 – 永豐銀行',
 'Green卡',
 'DAWHO 现金回饋信用卡 - 永豐銀行',
 'MasterCard (Foreigners Only) – SinoPac',
 '永豐銀行現金儲值卡 – 永豐銀行',
 '永豐財富 無限卡-永豐銀行',
 '永豐銀行',
 '永豐美麗華卡-永豐銀行',
 'JCB卡特選通路',
 '永豐銀行農騰卡 – 永豐銀行',
 'MITSUI OUTLET PARK聯名卡',
 'Me Card-永豐銀行',
 '永豐簽帳金融卡(興大分行限定版)',
 '永豐銀行']

In [61]:
import string

# Define Chinese punctuation characters


# Sample text with Chinese punctuation
text = "你好，世界！这是一个测试：你想要去除标点符号吗？"

# Create a translation table that maps each Chinese punctuation character to None
translator = str.maketrans('', '', chinese_punctuation)

# Apply the translation to remove punctuation
cleaned_text = text.translate(translator)

print(cleaned_text)

你好世界这是一个测试你想要去除标点符号吗？


### Summarize the documents

In [11]:
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import ChatOllama

llm = ChatOllama(model='qwen2:7b-instruct', temperature=0)
prompt_template = """以下文字為星展銀行信用卡的資訊，請幫我撰寫簡潔的摘要，能夠清楚明瞭是什麼信用卡、優惠有什麼、注意事項及活動時間等重要資訊。請勿解釋或是添加前言，直接回覆答案即可。
輸出格式請以純文字輸出，勿以markdown方式呈現。
信用卡所有資訊:
{text}
答案:
  """

prompt_template = """將以下內容簡潔地總結為繁體中文:
{text}
簡潔的總結:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "您的工作是製作最終的摘要\n"
    "我們已經提供了一個現有的摘要，直到某個點為止: {existing_answer}\n"
    "我們有機會根據以下更多的背景信息"
    "如果需要）來完善現有的摘要\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "根據新的訊息，請將原始摘要進行修改（如果需要的話）。"
    "如果訊息不有幫助，請返回原始摘要。"
)
refine_prompt = PromptTemplate.from_template(refine_template)

dbs_prompt = PromptTemplate(template=prompt_template, input_variables=["text"])

stuff_chain = load_summarize_chain(llm, chain_type="refine",
                                   question_prompt=prompt,
                                    refine_prompt=refine_prompt,
                                    # return_intermediate_steps=True,
                                    input_key="input_documents",
                                    output_key="output_text",)

try:
    print(stuff_chain.run([soup_documents['dbs'][3]]))
except Exception as e:
    print(
        "The code failed since it won't be able to run inference on such a huge context and throws this exception: ",
        e,
    )

信用卡产品介绍:

1. **万事达卡商务礼遇星展商务eco卡**:
   - 针对商界需求设计,提供商务优惠和支付便利。
   - 支持货款垫付、广告费用等多用途消费。
   - 享有网购保险服务,保障网购安全。

2. **万事达卡商务钛金卡**:
   - 商务等级信用卡,提供一系列商务礼遇和服务。
   - 持卡人可享受特定的网上购物保险权益。

3. **申办资格**:
   - 年龄18至75岁,年收入需达到新台币22万元以上。

4. **年费办法**:
   - 正卡年费为新台币3000元。
   - 持卡第二年起,满足特定条件可免缴当年年费。

5. **环保材质**:
   - 使用PLA聚乳酸材料,比传统塑料更天然、耐用且对环境影响较小。

6. **信用卡刷卡优惠**:
   - 详情请查询刷卡优惠说明页面。

7. **循环信用利率**:
   - 区间为5.99%至14.99%,基准日为民国113年6月28日。

8. **存款保险**:
   - 本机构已参加存款保险,提供一定的资金安全保障。


In [142]:
soup_documents['dbs'][3].metadata

{'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/eco-biz?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-hyperlink',
 'content_type': 'text/html',
 'title': '星展商務eco卡 | 星展銀行（台灣）',
 'description': '國內／國外一般消費 1.5% 無上限，日本／韓國／美洲／歐洲／新加坡享最優 5% 回饋',
 'language': None,
 'bank': '星展銀行'}

In [132]:
import mlflow
from langchain.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

splitter = RecursiveCharacterTextSplitter(chunk_size=400,
                                        length_function=len,
                                        is_separator_regex=True,
                                        chunk_overlap=50,
                                        separators=['。'])
banks = list(bank_urls.keys())
chunks = splitter.split_documents(soup_documents[banks[0]])
for b in banks[1:]:
    chunks.extend(splitter.split_documents(soup_documents[b]))

embedding = HuggingFaceEmbeddings(model_name='BAAI/bge-base-zh-v1.5', model_kwargs={'device':'mps'})
collection_name = 'credit_card'
qdrant_url = 'http://localhost:6333'
# vec_store = QdrantVectorStore.from_existing_collection(
#                               collection_name=collection_name,
#                               embedding = embedding,
#                               url = qdrant_url)

vec_store = QdrantVectorStore.from_documents(chunks,
                              collection_name=collection_name,
                              embedding = embedding,
                              force_recreate=True,
                              url = qdrant_url)



## Graph RAG

In [140]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm = ChatOllama(model='jack/llama3-8b-chinese')
llm_transformer = LLMGraphTransformer(llm=llm)

all_docs = soup_documents[list(soup_documents.keys())[0]]
for k in list(soup_documents.keys())[1:]:
    all_docs.extend(soup_documents[k])
# print(len(all_docs))

graph_documents = llm_transformer.convert_to_graph_documents(all_docs[:20])
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[]
Relationships:[]


## Self-querying retriever

In [143]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The url of the credit card webpage",
        type="string",
    ),
    AttributeInfo(
        name="title",
        description="The title of the credit card webpage",
        type="integer",
    ),
    AttributeInfo(
        name="description",
        description="The description of the credit card webpage",
        type="string",
    ),
    AttributeInfo(
        name="language",
        description="The language of the credit card webpage",
        type="string"
    ),
    AttributeInfo(
        name="bank",
        description="The bank that hosts the credit card webpage",
        type="string"
    ),
    AttributeInfo(
        name="content_type",
        description="The content type for a webpage refers to the MIME type or media type specified in the HTTP header, which tells the browser how to interpret the content, such as text/html for HTML documents, application/json for JSON data, or image/jpeg for JPEG images.",
        type="string"
    ),
]
document_content_description = "The content between HTML tags that is displayed on a credit card webpage, such as headings, paragraphs, links, and other textual elements"

retriever = SelfQueryRetriever.from_llm(
    llm,
    vec_store,
    document_content_description,
    metadata_field_info,
)

In [147]:
retriever.invoke("星展銀行信用卡優惠最高多少")

[Document(metadata={'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-pchomeprime/index.html?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-pchome-hyperlink', 'content_type': 'text/html', 'title': '星展PChome聯名卡 回饋太強 好刷到家 | 星展銀行(台灣)', 'description': '回饋太強 好刷到家！PChome站內消費天天4%，最高6%回饋（分期同享一筆）| 國内一般消費 0.5%，國外消費1.5% 聯名點數', 'language': None, 'bank': '星展銀行', '_id': '3f57e90c-eb0a-49b6-94d9-9e7b220ea4d8', '_collection_name': 'credit_card'}, page_content='。\n星展銀行僅提供各項優惠訊息，星展銀行並非商品或服務之出售人或提供人。所有廣告內容及圖片均由各參與廠商提供，且各優惠商品及服務均由參與商家直接提供予持卡人，若持卡人對服務內容或商品有任何爭議，請逕自與各廠商處理。\n星展銀行保留得隨時變更、暫停、取消、終止本活動之權利，如有任何變動，以本行網站之最新公告為準'),
 Document(metadata={'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-happygo/index.html?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-funlife-hyperlink', 'content_type': 'text/html', 'title': '【星展HAPPY GO聯名卡】指定通路享點數3倍回饋｜星展銀行', 'description': '不用精算．每筆都賺！刷卡享指定商店3倍、特約商店2倍、一般消費1倍點數回饋，立即申辦。', 'language': None, 'bank': '星展銀行', '_id': '923a9e47-ff14-45cc-a

In [13]:
from langchain.retrievers.multi_query import MultiQueryRetriever
retriever = vec_store.as_retriever(search_kwargs={'k':10, 'score_threshold':0.5})
    
llm = ChatOllama(model='qwen2:7b-instruct', temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=retriever, llm=llm
)


In [148]:
retriever_from_llm.invoke("星展銀行優惠最高是哪張信用卡？")

[Document(metadata={'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/classic?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-cta', 'content_type': 'text/html', 'title': '星展金卡/普卡 | 台灣星展銀行', 'description': '星展信用卡讓您聰明用卡，享盡精彩', 'language': None, 'bank': '星展銀行', '_id': '49b1335f-859f-4634-b376-08241afee773', '_collection_name': 'credit_card'}, page_content='星展金卡/普卡\n星展信用卡讓您聰明用卡，享盡精彩 (目前不提供新卡申請)\n主要特色現金回饋，無上限刷卡消費即時簡訊服務\n現金回饋，無上限最高0.5%現金回饋，您的每筆星展信用卡消費，皆可累計和回饋，回饋無上限，讓您輕鬆刷卡消費，歡欣享回饋！ 詳細活動內容說明如下：每月新增消費金額次月現金紅利回饋比率NT $100,000以上0.5%NT $50,000至NT $99,9990.3%NT $30,000至 NT $49,9990.1%NT $0至NT $29,9990%新增刷卡消費金額以實際入帳日為準。現金回饋方式除有明顯之計算錯誤外，當月新增消費金額以每月帳單上所載之新增消費金額為準，且正附卡刷卡消費金額合併計算。現金回饋專案所回饋之現金回饋扣抵次月起新增之消費金額，不得提領、折現或轉讓。此現金回饋可累積抵用'),
 Document(metadata={'source': 'https://www.dbs.com.tw/personal-zh/cards/dbs-credit-cards/classic?pid=tw-pweb-personal-zh_cards_dbs-credit-cards_default_page-hyperlink', 'content_type': 'text/html', 'title': '星展金卡/普卡 | 台灣星展銀行', 'description': '星展信用

In [19]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder 
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_core.runnables import RunnablePassthrough
import opencc

from opik.integrations.langchain import OpikTracer
import os
import opik
# opik.configure(use_local=True)

# Create the Opik tracer
# opik_tracer = OpikTracer(tags=["langchain", "ollama"])
# os.environ["OPIK_PROJECT_NAME"] = "credit-card-helper"
converter = opencc.OpenCC('s2t.json')
reranker = HuggingFaceCrossEncoder(model_name='BAAI/bge-reranker-base')
compressor = CrossEncoderReranker(model=reranker, top_n=10)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever_from_llm)

model = ChatOllama(model='qwen2:7b-instruct', temperature=0)
# .with_config({"callbacks": [opik_tracer]})

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI question-answering assistant. Your task is to answer the question based on the provided documents. The documents are part of the text from the description of the credit cards.
    The documents are not all relevant to the question. Please filter and reply with an answer. No pre-amble or explanation. Please output text only, not in markdown format.
    
    Documents: 
    {context}

    Question:
    {question}

    Answer:""",
)

def format_docs(docs):
    return "\n\n".join([f"DOCUMENT {index}\nTitle: {d.metadata['title']}\n{d.page_content}" for index, d in enumerate(docs)])


chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | QUERY_PROMPT
    | model
    | StrOutputParser()
)

question = "國泰銀行有什麼信用卡？"
response = chain.invoke(question)
# compression_retriever.invoke(question)
print(converter.convert(response))

國泰銀行有多款信用卡供選擇：

1. **世界卡**: 提供特定回饋優惠，但不包含某些費用和交易類型。
2. **聯合報優秀卡**: 已停發，提供年費、掛失費等各項費用及產品權益的資訊可參閱各產品網頁。
3. **NFC行動信用卡**: 已停發，不再受理申請與補發業務。

這些是國泰銀行提供的幾款信用卡類型。每張卡都有其特定的優點和適用條件，請根據自己的需求選擇適合的信用卡。


In [8]:
from opik.evaluation.metrics import Hallucination
from opik.evaluation.models import LiteLLMChatModel

model = LiteLLMChatModel(
    name="ollama/llama3.2",
    base_url="http://localhost:11434"
)

hallucination_metric = Hallucination(
    model=model
)

In [ ]:
hallucination_metric.score(
    input=question,
    output=response,
)

In [1]:
from litellm import completion

response = completion(
    model="ollama/llama3.2", 
    messages=[{ "content": "respond in 20 words. who are you?","role": "user"}], 
    api_base="http://localhost:11434"
)
print(response['choices'][0].message.content)


ModelResponse(id='chatcmpl-d7c53168-a465-429f-bfbb-6b957da8a265', created=1735090100, model='ollama/llama3.2', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="I'm an artificial intelligence designed to provide helpful and informative responses, assisting users with various inquiries and tasks efficiently.", role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=23, prompt_tokens=38, total_tokens=61, completion_tokens_details=None, prompt_tokens_details=None))
